In [1]:
import helper

import difflib 

import seaborn as sns
import numpy as np
import pandas as pd
import collections
import matplotlib.pyplot as plt

from scipy.stats.stats import pearsonr  
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# # Filter the data
# col_to_drop = ['weather','date_time','daily_mape','mape_std']
# col_to_drop = []

# east_data = east_data[(east_data.weather =='sunny')]
# west_data = west_data[(west_data.weather =='sunny')]
# lower_data = lower_data[(lower_data.weather =='sunny')]

# east_data = east_data[east_data.weather != 'sunny'].drop(col_to_drop,axis=1)
# west_data = west_data[west_data.condition == 'normal'].drop(col_to_drop,axis=1)
# lower_data = lower_data[lower_data.condition == 'normal'].drop(col_to_drop,axis=1)

In [1]:
# # Define dataset
# dataset = east_data
# daily_mape = process.daily_dict(dataset,'daily_mape')
# daily_weather = process.daily_dict(dataset,'weather')
# daily_std = process.daily_dict(dataset,'mape_std')
# daily_empty = process.daily_dict(dataset)

In [2]:
# # Daily MAPE histogram
# visual.histogram(list(east_data['weather'].values),"east_hist",False,500)
# visual.histogram(list(west_data['daily_mape'].values),"west_hist",True,500)
# visual.histogram(list(lower_data['daily_mape'].values),"south_hist",True,500)

In [3]:
# # Weather Pie Chart
# weathers = []
# for _, i in daily_weather.items():
#     weathers.append(i)
# visual.pie_chart(weathers,"Weather Condition - 860 Days",True,500)

In [4]:
# # Predict power output by regression and reture the predictions for each panel
# res1,pred1 = model.model_predict(east_data,True)
# res2,pred2 = model.model_predict(west_data,True)
# res3,pred3 = model.model_predict(lower_data,True)

# # Convert predictions into DataFrame format for each panel
# df = pd.DataFrame((pred1[0]))
# for i in range(1,len(pred1)):
#     df[str(i)] = pd.DataFrame(pred1[i])
    
# # Save the predictions 
# export_csv = df.to_csv ("east_normal_pred.csv", index = False, header=True) 

In [5]:
# # Manually calculate MAPE and find distribution
# dataset = pd.read_csv('east_normal_gt_pred.csv')

# n = len(dataset.columns)//2
# errors = {'MAPE':[],'MSE':[],'MAE':[]}

# error_high = []

# for i in range(n):
#     col = 'panel_'+str(i+1)
#     gt = dataset[col+'_power']
#     pred = dataset[col+'_predict']
#     error = []
#     high_error_count = 0
    
#     for j in range(len(gt)):
#         err = np.abs(gt[j]-pred[j])/gt[j]
#         if err == np.float64('inf') or err == np.float64('nan'):
#             continue
#         error.append(np.abs(gt[j]-pred[j])/gt[j])
        
#         if err > 0.05:
#             high_error_count += 1
#     error_high.append(high_error_count)

#     errors['MAPE'].append(np.nansum(error)/len(error)*100)
#     errors['MSE'].append(mean_squared_error(gt,pred))
#     errors['MAE'].append(mean_absolute_error(gt,pred))

In [2]:
# for day in set(wrong_date):
#     data = daily_data[day].drop(['weather','daily_mape','mape_std','condition'],axis=1)
#     mape = round(df[df['date']==day]['daily_mape'],2)
#     std = round(df[df['date']==day]['mape_std'],2)
#     corr = round(df[df['date']==day]['correlation'],3)
#     weather = df[df['date']==day]['weather']
    
# #     title = "MAPE "+str(mape)+"; STD "+str(std)+"; corr "+str(corr)+"; weather " + weather
#     title = "MAPE "+str(mape)
#     visual.raw_data_plot(data,title,save=False,resolution=300)

In [33]:
# miss_data = ['20190829','20190930','20190905','20190906','20190917','20190919','20191010','20191017',
#              '20191031','20191120','20191125','20191213','20190223','20191012','20180516','20180820',
#              '20190510','20170920','20190824','20190827','20180829']

# for miss in miss_data:
#     data = raw_data[miss].drop(['weather','daily_mape','mape_std','condition'],axis=1)
#     visual.raw_data_plot(data,miss)
#     plt.show()

In [3]:
# # Create a new dataframe from old one
# df = pd.DataFrame()
# for day in raw_data:
#     data = raw_data[day]
#     data["date"] = day
#     df = pd.concat([df,data], ignore_index=True)

In [2]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier


from sklearn.svm import SVC

def create_dataset(data):
        
    dataX, dataY = [],[]
    for i in range(len(data)):
        dataX.append(data[i][:-1])
        dataY.append(int(data[i][-1])) 
            
    return np.array(dataX), np.array(dataY)

def random_forest_class(dataset):
    
    date = dataset['date'].values
    dataset = dataset.drop(['date'],axis=1).values
        
    X, Y = create_dataset(dataset)
    cvscores = []
    cm = []
    kfold = StratifiedKFold(n_splits=3, shuffle=True)
        
    for train_index, test_index in kfold.split(X, Y):
            
        X_train = X[train_index]
        y_train = Y[train_index]
        X_test = X[test_index]
        y_test = Y[test_index]
        test_date = date[test_index]
        
        rf_model = RandomForestClassifier(max_depth=10, random_state=1).fit(X_train, y_train)
        rf_prediction = rf_model.predict(X_test)
        cvscores.append(accuracy_score(y_test, rf_prediction)*100)
        cm.append(confusion_matrix(y_test, rf_prediction))
        
    return [cvscores, cm, rf_model,X_test,y_test,test_date]

def confusion_matrix_plot(cm,acc,save=False):
    
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(1,1,1) 
    sns.set(font_scale=1)
    sns.heatmap(cm, annot=True,fmt='g',annot_kws={"size": 15},cmap="Blues", square=True, cbar=False)

    # labels, title and ticks
    ax.set_xlabel('Prediction',fontsize=18)
    ax.set_ylabel('Ground Truth',fontsize=18)
    title_name = 'Confusion Matrix - Accuracy ' + str(round(acc,2)) + "%"
    ax.set_title(title_name,fontsize=20) 
    ax.xaxis.set_ticklabels(['Normal', 'Anomaly'],rotation=45)
    ax.yaxis.set_ticklabels(['Normal', 'Anomaly'],rotation=45)

    plt.tight_layout()
    if save is True:
        plt.savefig(title_name+'.jpg', format='jpg', dpi=1000)
    plt.show()

In [104]:
# Update labels
feature = pd.read_csv('new_features.csv')
east_data = pd.read_csv('new_west_data.csv')

feature_day_cond = {}

new_cond = []
for i in feature.index:
    feature_day_cond[str(int(feature.loc[i]['date']))] = int(feature.loc[i]['cond'])

for j in east_data.index:
    day = str(east_data.loc[j]['date_time'])
    condition = east_data.loc[j]['condition']
    
    if day not in feature_day_cond:
        new_cond.append(0)
        continue
    new_cond.append(feature_day_cond[day])
    
east_data['condition'] = new_cond
export_csv = east_data.to_csv ("new_west_data.csv", index = False, header=True)

In [10]:
# Load dataset
# feature = pd.read_csv('new_features.csv')
east_data = pd.read_csv('new_west_data.csv')

# Create object
visual = helper.visualizer()
model = helper.modeling()
process = helper.data_processing()

# Create daily raw data
raw_data = process.mape_distribution(east_data)

In [12]:
from scipy.stats.stats import pearsonr  

weather_collect = collections.Counter(east_data['weather']).most_common()
weather_dict = {}
for ith, weather_info in enumerate(weather_collect):
    weather_dict[weather_info[0]] = ith

feature_dict = {'date':[],'mse':[],'std':[],'corr':[],'max_power':[],'weather':[],'cond':[]}

for day in raw_data:
    
    if len(raw_data[day]) < 20:
        continue
    
    feature_dict['date'].append(day)
    feature_dict['weather'].append(weather_dict[raw_data[day]['weather'].values[0]])
    data = raw_data[day].drop(['weather','condition'],axis=1)
    
    # MSE and STD features
    res,_ = model.model_predict(data)
    feature_dict['mse'].append(np.mean(res))
    feature_dict['std'].append(np.std(res))
    
    # Correlation feature
    res = []
    first = 'panel_1_power'
    for col in data.columns:
        if col != first:
            res.append(pearsonr(data[col].values,data[first].values)[0])
    
    feature_dict['corr'].append(min(res))
    
    # Max power feature
    power = []
    for col in data.columns:
        power.append(max(data[col]))
    
    feature_dict['max_power'].append(np.mean(power))
    
    # Condition
    condition = east_data[east_data['date_time']==float(day)]['condition'].values[0]
    feature_dict['cond'].append(condition)
    

In [80]:
# df = pd.DataFrame()

# for key,value in feature_dict.items():
#     df[key] = value

dataset = pd.read_csv('new_west_data.csv')   
df = process.feature_generate(dataset)
    
score, cm, rf_model,X_test,y_test,test_date = random_forest_class(df.dropna())
print(score)

wrong_day = []
predictions = rf_model.predict(X_test)

for i, pred in enumerate(predictions):
    if y_test[i] != pred:
        wrong_day.append(test_date[i]) 
        
check_day = pd.DataFrame()
for day in wrong_day:
    check_day = pd.concat([check_day,df[df['date']==day]], ignore_index=True)

[88.16793893129771, 88.54961832061069, 85.38461538461539]


In [81]:
export_csv = df.to_csv ("new_features.csv", index = False, header=True)

In [102]:
feature = pd.read_csv('new_features.csv')

score, cm, rf_model,X_test,y_test,test_date = random_forest_class(feature.dropna())
print(score)

wrong_day = []
predictions = rf_model.predict(X_test)

for i, pred in enumerate(predictions):
    if y_test[i] != pred:
        wrong_day.append(test_date[i]) 
        
check_day = pd.DataFrame()
for day in wrong_day:
    check_day = pd.concat([check_day,feature[feature['date']==day]], ignore_index=True)

[95.41984732824427, 92.72030651340997, 90.80459770114942]


In [111]:
for i in check_day.index:
    error_ = round(check_day.loc[i]['mse'],5)
    std_ = round(check_day.loc[i]['std'],5)
    corr_ = round(check_day.loc[i]['corr'],5)
    condition = check_day.loc[i]['cond']
    day = str(int(check_day.loc[i]['date']))
    data = raw_data[day].drop(['weather','condition'],axis=1)
    
    title ='Date: ' + str(day) + ' MSE: ' + str(error_) + ' STD: ' + str(std_) + ' Corr: ' + str(corr_) + ' Condition: ' + str(condition)
    visual.raw_data_plot(data,title,True)

KeyError: 'mse'

In [65]:
collections.Counter(feature[feature['corr']>=0.96]['cond'])

Counter({1: 196, 0: 6})

In [110]:
check_day = dataset[dataset['weather']=='sunny'][dataset['condition']==0]